In [1]:
import pandas as pd
import numpy as np
import time
from alpha_vantage.timeseries import TimeSeries

# read SP500 components 
df_tickers = pd.read_csv("sp500_components.csv")
df_tickers = df_tickers.rename(columns={"Unnamed: 0": "comp_index"})

# creating a list of all tickers
melted_tickers = pd.melt(df_tickers, id_vars="comp_index", var_name="year", value_name="ticker").dropna()
tickers = melted_tickers["ticker"].unique().tolist()

# api_key 
api_key1 = "XQ40GSMB7OJOL6VQ"
api_key2 = "5X3HSKBVP8LT7590"
api_key3 = "V1PFJOMDHX76TFF8"

# download the adjusted close price for each ticker
data_frames = []
missing_symbols = []

for ticker in tickers:
    try:
        # use api_key1 before exceding number of requests in one day
        ts = TimeSeries(key=api_key1, output_format='pandas')

        print(tickers.index(ticker), ticker)

        data, meta_data = ts.get_daily_adjusted(symbol=ticker, outputsize='full')
        # rename the column and select only the adjusted close price
        data = data[["5. adjusted close"]].rename(columns={"5. adjusted close":"Adj_Close"})
        # create ticker column
        data["ticker"] = ticker
        # get log-returns
        data["log_rtn"] = np.log(data["Adj_Close"]).diff().fillna(0)
        # sort the index in ascending order
        data = data.sort_index()
        # add data to data_frames list 
        data_frames.append(data)
    except ValueError as e:
        error_message = str(e)
        if "Invalid API call" in error_message:
            print(f"Error processing {ticker}: {error_message}")
            # append to removed symbols list
            missing_symbols.append(ticker)
            # do not append the ticker back to the list
            continue
        else:
            print(f"Error processing {ticker}: {error_message}")
            # append the ticker back to the list
            tickers.append(ticker)
            # wait 1 minute before trying again
            time.sleep(60)  
    except ConnectionError as e:
        # use api_key2 before exceding number of requests in one day
        print(f"Error processing {api_key1}: {error_message}")
        # append the ticker back to the list
        tickers.append(ticker)
            
        # new ts
        ts = TimeSeries(key=api_key2, output_format='pandas')

        print(tickers.index(ticker), ticker)

        try:
            data, meta_data = ts.get_daily_adjusted(symbol=ticker, outputsize='full')
            # rename the column and select only the adjusted close price
            data = data[["5. adjusted close"]].rename(columns={"5. adjusted close":"Adj_Close"})
            # create ticker column
            data["ticker"] = ticker
            # get log-returns
            data["log_rtn"] = np.log(data["Adj_Close"]).diff().fillna(0)
            # sort the index in ascending order
            data = data.sort_index()
            # add data to data_frames list 
            data_frames.append(data)
        except ValueError as e:
            error_message = str(e)
            if "Invalid API call" in error_message:
                print(f"Error processing {ticker}: {error_message}")
                # append to removed symbols list
                missing_symbols.append(ticker)
                # do not append the ticker back to the list
                continue
            else:
                print(f"Error processing {ticker}: {error_message}")
                # append the ticker back to the list
                tickers.append(ticker)
                # wait 1 minute before trying again
                time.sleep(60)  
        except ConnectionError as e:
             # use api_key3 before exceding number of requests in one day
            print(f"Error processing {api_key2}: {error_message}")
            # append the ticker back to the list
            tickers.append(ticker)
                
            # new ts
            ts = TimeSeries(key=api_key3, output_format='pandas')

            print(tickers.index(ticker), ticker)

            try:
                data, meta_data = ts.get_daily_adjusted(symbol=ticker, outputsize='full')
                # rename the column and select only the adjusted close price
                data = data[["5. adjusted close"]].rename(columns={"5. adjusted close":"Adj_Close"})
                # create ticker column
                data["ticker"] = ticker
                # get log-returns
                data["log_rtn"] = np.log(data["Adj_Close"]).diff().fillna(0)
                # sort the index in ascending order
                data = data.sort_index()
                # add data to data_frames list 
                data_frames.append(data)
            except ValueError as e:
                error_message = str(e)
                if "Invalid API call" in error_message:
                    print(f"Error processing {ticker}: {error_message}")
                    # append to removed symbols list
                    missing_symbols.append(ticker)
                    # do not append the ticker back to the list
                    continue
                else:
                    print(f"Error processing {ticker}: {error_message}")
                    # append the ticker back to the list
                    tickers.append(ticker)
                    # wait 1 minute before trying again
                    time.sleep(60)  
            except ConnectionError as e:
                # use api_key3 before exceding number of requests in one day
                print(f"Error processing {api_key3}: {error_message}")

# concatenate the data frames into a single data frame
combined_data = pd.concat(data_frames, axis=0)

adj_close_data = combined_data.loc[:, ["ticker", "Adj_Close"]].reset_index().pivot(index="date", columns="ticker", values="Adj_Close")
log_rtn_data = combined_data.loc[:, ["ticker", "log_rtn"]].reset_index().pivot(index="date", columns="ticker", values="log_rtn")

# print tickers not found
missing_symbols_df = pd.DataFrame({"Symbols": missing_symbols})
print(missing_symbols)

# print the first and last 5 rows of the data 
print(adj_close_data.head())
print(log_rtn_data.tail())

# save csv file
missing_symbols_df.to_csv("missing_symbols.csv")
adj_close_data.to_csv("adj_close.csv")
log_rtn_data.to_csv("log_rtn.csv")


0 AAL
1 AAMRQ
2 AAPL
3 ABI
4 ABS
Error processing ABS: Invalid API call. Please retry or visit the documentation (https://www.alphavantage.co/documentation/) for TIME_SERIES_DAILY_ADJUSTED.
5 ABT
6 ABX
Error processing ABX: Invalid API call. Please retry or visit the documentation (https://www.alphavantage.co/documentation/) for TIME_SERIES_DAILY_ADJUSTED.
7 ACKH
Error processing ACKH: Invalid API call. Please retry or visit the documentation (https://www.alphavantage.co/documentation/) for TIME_SERIES_DAILY_ADJUSTED.
8 ACV
Error processing ACV: Thank you for using Alpha Vantage! Our standard API call frequency is 5 calls per minute and 500 calls per day. Please visit https://www.alphavantage.co/premium/ if you would like to target a higher API call frequency.
9 ADM
10 ADP
11 ADSK
12 AEE
13 AEP
14 AET
Error processing AET: Thank you for using Alpha Vantage! Our standard API call frequency is 5 calls per minute and 500 calls per day. Please visit https://www.alphavantage.co/premium/ if 